In [159]:
import gc
import re
from collections import defaultdict
from pprint import pprint
from time import time

import faiss
import numpy as np
import pandas as pd
from sentence_transformers import CrossEncoder, SentenceTransformer, util
from tqdm import tqdm
from transformers import pipeline

## Датасет

In [160]:
df = pd.read_csv(
    "/content/drive/MyDrive/resume_job_matching/Raw_Jobs.csv", delimiter=";"
)
df.head()

,№,id,title,salary,experience,job_type,description,key_skills,company,location,date_of_post,type
0,1,78577559,Главный механик,от 160000 до 160000 RUR,Более 6 лет,"Полная занятость,полный день","Вакансия компании: ООО ПК Предприятие ""ПИК"" ...",NaN,JCat.ru,Владивосток,27.03.2023,close
1,2,78693069,HTML-верстальщик,з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",В креативное агентство на полный рабочий день ...,NaN,Чугунова Ирина,NaN,29.03.2023,close
2,3,78945452,Тренер (направление профессионального обучения),з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",Наши предложения: самостоятельность в...,NaN,Перекрёсток,NaN,18.04.2023,close
3,4,79352008,Старший механик на КСПГ,з/п не указана,Более 6 лет,"Полная занятость,полный день",Обязанности: _Обеспечение технически ...,NaN,Газпром гелий сервис,NaN,17.04.2023,close
4,5,79406472,System Analyst Trainee,з/п не указана,Нет опыта,"Стажировка,полный день",IT-компания Aston - компания по разработке про...,"UML,SQL,Scrum,Retail,Базы данных,Kanban,Waterf...",Aston,NaN,17.04.2023,close


In [161]:
valid_comment_pattern = r"программист|разработчик|developer"

In [162]:
df["is_valid_pattern"] = df["title"].apply(
    lambda x: True
    if not isinstance(x, float)
    and re.match(valid_comment_pattern, x.lower()) is not None
    else False
)

In [163]:
df = df[df.is_valid_pattern == True]

In [164]:
def extract_information(job_description):
    responsibilities_pattern = r"\b(?:вам предстоит|вы будете|задачи|обязанности|чем необходимо заниматься)\b:(.*?)(?=\b(?:требования|требования к кандидатам|мы предлагаем|условия|требования:|мы ожидаем|наши пожелания|будет плюсом|будет преимуществом)\b|$)"
    requirements_pattern = r"\b(?:для нас важно|ждем от вас|мы ожидаем|наши пожелания|необходимые навыки|требования|требования к кандидатам|будет плюсом|будет преимуществом)\b:(.*?)(?=\b(?:мы предлагаем|условия|место работы|длительность рабочего дня)\b|$)"
    conditions_pattern = r"\b(?:мы предлагаем|условия)\b:(.*?)(?=\b(?:в ответ на оставленный отклик|образование)\b|$)"

    responsibilities_match = re.search(
        responsibilities_pattern, job_description, re.IGNORECASE | re.DOTALL
    )
    requirements_match = re.search(
        requirements_pattern, job_description, re.IGNORECASE | re.DOTALL
    )
    conditions_match = re.search(
        conditions_pattern, job_description, re.IGNORECASE | re.DOTALL
    )

    responsibilities = (
        responsibilities_match.group(1).strip() if responsibilities_match else None
    )
    requirements = requirements_match.group(1).strip() if requirements_match else None
    conditions = conditions_match.group(1).strip() if conditions_match else None

    return responsibilities, requirements, conditions

In [165]:
responsabilities, req, cond = list(), list(), list()

In [166]:
for row in tqdm(df["description"]):
    responsability, requirements, conditions = extract_information(row)
    responsabilities.append(responsability)
    req.append(requirements)
    cond.append(conditions)

100%|██████████| 1219/1219 [00:00<00:00, 2439.56it/s]


In [167]:
df["responsabilities"] = responsabilities
df["requirements"] = req
df["conditions"] = cond

In [168]:
del responsabilities
del req
del cond
gc.collect()

265

In [169]:
vacancy_text = defaultdict(str)
for idx, row in df.iterrows():
    vacancy_text[
        idx
    ] = f"Профессия: {row['title']}. Ключевые навыки: {row['key_skills']}. Обязанности: {row['responsabilities']}. Требования: {row['requirements']}. Условия {row['conditions']}"

In [170]:
df["text"] = df.index.map(vacancy_text)

# Алгоритм работы поиска резюме по вакансии

Алгоритм состоит из 2 шагов:

1. Превращаем текста в векторы с помощью энкодера и загружаем его в faiss. В качестве энкодера будем использовать intfloat/multilingual-e5-large
2. Делаем реранжирование с помощью кросс-энкодера

In [171]:
model = SentenceTransformer("intfloat/multilingual-e5-large")

In [172]:
query = """Программист C++, Web-программист, разработчик аппаратуры,.Опыт работы 5 лет 8 месяцев  Март 2019 — по настоящее время 3 месяца Удаленная работа Обнинск Web-программист Доработка сайта www.autolarek24.ru  Октябрь 2013 — по настоящее время 5 лет 8 месяцев АО «ГНЦ РФ – ФЭИ» Обнинск , www.ippe.ru Энергетика ... Атомная энергетика (генерация электроэнергии, АЭС) Инженер-программист Разработка ПО для операционных систем жесткого реального времени. Разработка драйверов, многопоточных приложений, клиент-серверных приложений. Разработка микропроцессорных систем. Работа с промышленными интерфейсами и протоколами: ModBus, CAN, RS422/485. Разработка документации ПО. Работа с измерительной аппаратурой с очень высокой точностью измерения и написание ПО для мониторинга и фильтрации измерений, а так же участие в разработке, настройке и тестировании данной аппаратуры.  Июнь 2017 — Ноябрь  2017 6 месяцев ООО "Персона" Обнинск Информационные технологии, системная интеграция, интернет ... Разработка программного обеспечения Веб-разработчик Разработка веб-сайтов, концепция MVC, командная разработка (Git), Laravel. Проект www.combeep.com"""

In [173]:
encoded_data = model.encode(df.text.tolist())
encoded_data = np.asarray(encoded_data.astype("float32"))
index = faiss.IndexIDMap(faiss.IndexFlatIP(1024))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, "vacancy.index")

## Faiss - отбираем кандидатов

In [174]:
def fetch_vacancy_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = dict()
    meta_dict["title"] = info["title"]
    meta_dict["text"] = info["text"][:1024]
    meta_dict['vacancy_idx'] = dataframe_idx
    return meta_dict


def search(query, top_k, index, model):
    t = time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print(">>>> Results in Total Time: {}".format(time() - t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results = [fetch_vacancy_info(idx) for idx in top_k_ids]
    return results

In [175]:
results = search(query, top_k=5, index=index, model=model)
print("\n")
for result in results:
    print("\t", result)

>>>> Results in Total Time: 0.06160378456115723


	 {'title': 'Программист-разработчик', 'text': 'Профессия: Программист-разработчик. Ключевые навыки: Python,Работа с базами данных,C++,Алгоритмы и структуры данных. Обязанности: разрабатывать оборудование и по для двухстороннего стрима данных в последовательных и распределенных сетях, тренировка алгоритмов под текущие нужды.   Рассматриваются только кандидаты для оффлайн работы   Договор, пятидневка с 9 до18 центральных район. Требования: опыт работы с микроконтроллерами, опыт самостоятельной разработки, электроника   Обязанности: разрабатывать оборудование и по для двухстороннего стрима данных в последовательных и распределенных сетях, тренировка алгоритмов под текущие нужды.   Рассматриваются только кандидаты для оффлайн работы   Договор, пятидневка с 9 до18 центральных район. Условия None', 'vacancy_idx': 115}
	 {'title': 'Программист C/С++', 'text': 'Профессия: Программист C/С++. Ключевые навыки: Linux,C/C++,C++,CMake,Git,Boost,doct

## 2. Cross Encoder

In [176]:
cross_model = CrossEncoder("PitKoro/cross-encoder-ru-msmarco-passage", max_length=2048)

In [177]:
def cross_score(model_inputs):
    scores = cross_model.predict(model_inputs)
    return scores

In [178]:
model_inputs = [[query, item["text"]] for item in results]
scores = cross_score(model_inputs)
# Sort the scores in decreasing order
ranked_results = [
    {"Title": inp["title"], "Text": inp["text"], "Score": score}
    for inp, score in zip(results, scores)
]
ranked_results = sorted(ranked_results, key=lambda x: x["Score"], reverse=True)

In [179]:
print("\n")
for result in ranked_results:
    print("\t", pprint(result))



{'Score': 0.49219885,
 'Text': 'Профессия: Программист С++. Ключевые навыки: Qt,Debian,Astra '
         'Linux,SQL,STL,Multithread Programming,MS Visual C++,OPC,TCP/IP. '
         'Обязанности: .      Разработка специализированного '
         'кросс-платформенного клиент-серверного ПО в среде QT. Требования: '
         'Обязательно:        опыт разработки ПО на базе клиент-серверной '
         'архитектуры под Linux     опыт разработки с использованием QT     '
         'практические навыки написания многопоточного кода     '
         'Желательно:        опыт разработки под Windows (MS Visual C++)     '
         'знание протоколов OPC UA/DA, MODBUS, SNMP, MEC 870-5-101/104      '
         'знание основных сетевых протоколов, технологий, шифрования (TCP/IP, '
         'UDP, ARP, ICMP, HTTP, HTTPS)      опыт написания библиотек     '
         'знание winapi, SQL, PostgreSQL, MS SQL Server, ODBC      английский '
         'язык на уровне чтения документации.. Условия Трудоустройство по 

## Метрики

Посчитаем метрики на размеченном датасете для IT вакансий

In [180]:
labeling = pd.read_csv("/content/drive/MyDrive/resume_job_matching/labeling.csv")

In [181]:
top100_vacancies = pd.read_csv('/content/drive/MyDrive/resume_job_matching/random100vac.csv')
top100_vacancies.head()

,№,id,Должность,Зарплата,Опыт работы,Занятость,Описание вакансии,Ключевые навыки,Компания,Город,Обновление резюме,Тип,is_valid_pattern,text
0,394,79011638,Разработчик Kotlin,з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день","Marlerino Group – молодая, развивающаяся I...","Java,Мобильные приложения,ООП,Kotlin,Android S...",Marlerino Group,Россия,06.04.2023,close,True,Разработчик Kotlin. Marlerino Group – моло...
1,426,79280145,Разработчик Bitrix24,з/п не указана,От 3 до 6 лет,"Полная занятость,полный день",Обязанности: Техническая поддержка П...,ключевые навыки отсутствуют,Медицинские товары,Россия,13.04.2023,close,True,Разработчик Bitrix24. Обязанности: Т...
2,1233,79472031,Программист 1С (начальный уровень),от 60000 RUR,От 1 года до 3 лет,"Полная занятость,полный день","В компанию, имеющую аккредитацию ИТ компаний, ...","1С: Зарплата и управление персоналом,1С: Бухга...",МАКСИМА,Россия,18.04.2023,close,True,Программист 1С (начальный уровень). В компанию...
3,1429,79600291,Программист станков с ЧПУ,от 35000 RUR,От 1 года до 3 лет,"Полная занятость,полный день",Обязанности: Программирование станков...,ключевые навыки отсутствуют,КНИИТМУ,"Калуга, улица Карла Маркса, 4",20.04.2023,close,True,Программист станков с ЧПУ. Обязанности: ...
4,2336,78562452,Разработчик / аналитик Qlik Sense,от 130000 до 150000 RUR,От 3 до 6 лет,"Полная занятость,полный день",Задачи: • Участие в формировании стратеги...,"Qlik Sense,BI,QlikView,Анализ бизнес показател...",А-Контракт,Россия,27.03.2023,close,True,Разработчик / аналитик Qlik Sense. Задачи: ...


In [182]:
top100_resume = pd.read_csv('/content/drive/MyDrive/resume_job_matching/random100resume.csv')
top100_resume.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,is_valid_pattern,text
0,"Мужчина , 27 лет , родился 8 августа 1991",70000 руб.,"Программист C++, Web-программист, разработчик ...","Обнинск , готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день, сменный график, уд...",Опыт работы 5 лет 8 месяцев Март 2019 — по на...,Удаленная работа,Web-программист,Высшее образование 2018 ИАТЭ НИЯУ МИФИ Институ...,17.04.2019 16:49,Имеется собственный автомобиль,True,"Программист C++, Web-программист, разработчик ..."
1,"Мужчина , 24 года , родился 14 июля 1994",70000 руб.,Программист,"Тюмень , не готов к переезду , не готов к кома...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы 3 года 2 месяца Декабрь 2018 — по...,Индивидуальное предпринимательство / частная п...,Программист,Высшее образование 2017 НГИИ (бывш. НИИ) Факул...,13.05.2019 12:01,Не указано,True,Программист.Опыт работы 3 года 2 месяца Декаб...
2,"Женщина , 44 года , родилась 15 октября 1974",50000 руб.,Программист,"Губкин , готова к переезду , готова к командир...",полная занятость,"полный день, сменный график, удаленная работа",Опыт работы 12 лет 5 месяцев Декабрь 2016 — А...,"ООО""ТД Молочный мир""",Торговый представитель,Высшее образование 2011 Московский Государстве...,12.04.2019 12:54,Не указано,True,Программист.Опыт работы 12 лет 5 месяцев Дека...
3,"Мужчина , 24 года , родился 13 июня 1994",30000 руб.,Программист-разработчик,"Ростов-на-Дону , готов к переезду , готов к ко...",полная занятость,полный день,Опыт работы 2 года 11 месяцев Июль 2016 — по ...,"ООО ""АПС""",Инженер,Высшее образование (Магистр) 2018 Ростовский ...,12.04.2019 11:52,Не указано,True,Программист-разработчик.Опыт работы 2 года 11 ...
4,"Мужчина , 35 лет , родился 12 марта 1984",1800 руб.,Программист 1С 8,"Москва , м. Выхино , не готов к переезду , не...","частичная занятость, проектная работа","гибкий график, удаленная работа",Опыт работы 16 лет 2 месяца Декабрь 2007 — по...,Комплексное сопровождение клиентов по автомати...,Программист 1С,Высшее образование 2008 Южно-Российский госуда...,07.05.2019 11:34,Не указано,True,Программист 1С 8.Опыт работы 16 лет 2 месяца ...


In [183]:
labeling.head()

,Unnamed: 0,is_relevant,score,interpretation,resume_index,vacancy_index
0,0,0,2,"Вакансия не подходит, поскольку опыт работы в ...",83,4
1,1,0,0,"Вакансия не соответствует опыту кандидата, так...",83,63
2,2,0,0,"Вакансия не релевантна, поскольку требуется сп...",83,3
3,3,0,0,"Отсутствие в резюме упоминания о знании PHP, L...",83,36
4,4,0,2,Не подходит: опыт работы соискателя значительн...,83,44


In [184]:
vacancies = top100_vacancies[top100_vacancies.index.isin(labeling.vacancy_index.unique())]
vacancies.head()

,№,id,Должность,Зарплата,Опыт работы,Занятость,Описание вакансии,Ключевые навыки,Компания,Город,Обновление резюме,Тип,is_valid_pattern,text
1,426,79280145,Разработчик Bitrix24,з/п не указана,От 3 до 6 лет,"Полная занятость,полный день",Обязанности: Техническая поддержка П...,ключевые навыки отсутствуют,Медицинские товары,Россия,13.04.2023,close,True,Разработчик Bitrix24. Обязанности: Т...
3,1429,79600291,Программист станков с ЧПУ,от 35000 RUR,От 1 года до 3 лет,"Полная занятость,полный день",Обязанности: Программирование станков...,ключевые навыки отсутствуют,КНИИТМУ,"Калуга, улица Карла Маркса, 4",20.04.2023,close,True,Программист станков с ЧПУ. Обязанности: ...
4,2336,78562452,Разработчик / аналитик Qlik Sense,от 130000 до 150000 RUR,От 3 до 6 лет,"Полная занятость,полный день",Задачи: • Участие в формировании стратеги...,"Qlik Sense,BI,QlikView,Анализ бизнес показател...",А-Контракт,Россия,27.03.2023,close,True,Разработчик / аналитик Qlik Sense. Задачи: ...
5,2350,79071089,Разработчик фронтенда в команду API Карт,з/п не указана,От 3 до 6 лет,"Полная занятость,полный день",Мы разрабатываем JavaScript API Яндекс Карт. Н...,"JavaScript,TypeScript,React,API",Яндекс,"Москва, Садовническая улица, 82с2",09.04.2023,close,True,Разработчик фронтенда в команду API Карт. Мы р...
6,2595,78050064,Программист -тестировщик (отдел автотестирования),от 46000 до 92000 RUR,От 1 года до 3 лет,"Полная занятость,полный день",Тензор - это аккредитованная IT-компания: боле...,ключевые навыки отсутствуют,Тензор,Россия,10.04.2023,close,True,Программист -тестировщик (отдел автотестирован...


In [185]:
resumes  = top100_resume[top100_resume.index.isin(labeling.resume_index.unique())]
resumes.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,is_valid_pattern,text
1,"Мужчина , 24 года , родился 14 июля 1994",70000 руб.,Программист,"Тюмень , не готов к переезду , не готов к кома...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы 3 года 2 месяца Декабрь 2018 — по...,Индивидуальное предпринимательство / частная п...,Программист,Высшее образование 2017 НГИИ (бывш. НИИ) Факул...,13.05.2019 12:01,Не указано,True,Программист.Опыт работы 3 года 2 месяца Декаб...
6,"Мужчина , 22 года , родился 27 октября 1996",100000 руб.,Программист С++,"Москва , м. Войковская , не готов к переезду ...","проектная работа, частичная занятость","удаленная работа, гибкий график",Опыт работы 1 год 2 месяца Программист С++ 10...,Design Dossier,Программист,Неоконченное высшее образование 2020 Московск...,22.04.2019 16:06,Не указано,True,Программист С++.Опыт работы 1 год 2 месяца Пр...
26,"Мужчина , 48 лет , родился 17 октября 1970",30000 руб.,"Программист, оператор ПК","Санкт-Петербург , м. Парк Победы , готов к пе...","проектная работа, стажировка, частичная занято...","удаленная работа, гибкий график, полный день, ...",Опыт работы 23 года 6 месяцев Март 1995 — Авг...,ГК Бюллетень Недвижимости,Программист,Неоконченное высшее образование 1992 Санкт-Пе...,07.05.2019 21:02,Не указано,True,"Программист, оператор ПК.Опыт работы 23 года 6..."
30,"Мужчина , 36 лет , родился 14 февраля 1983",60000 руб.,Программист-разработчик,"Москва , м. Алтуфьево , готов к переезду (Бел...","частичная занятость, проектная работа, полная ...",удаленная работа,Опыт работы 9 лет 10 месяцев Август 2018 — Ма...,"ОАО ""Вента""",Программист,Высшее образование 2005 Уральский федеральный...,17.04.2019 15:13,Не указано,True,Программист-разработчик.Опыт работы 9 лет 10 м...
33,"Мужчина , 31 год , родился 21 октября 1987",50000 руб.,Разработчик ПО,"Набережные Челны , готов к переезду , готов к ...","частичная занятость, проектная работа, полная ...","полный день, сменный график, вахтовый метод, у...",Опыт работы 6 лет 2 месяца Разработчик ПО 50 ...,БАРС Груп,Web-разработчик,Высшее образование (Бакалавр) 2017 Филиал Каз...,22.04.2019 08:59,Не указано,True,Разработчик ПО.Опыт работы 6 лет 2 месяца Раз...


Построим модель на датасете резюме

In [186]:
encoded_data = model.encode(vacancies.text.tolist())
encoded_data = np.asarray(encoded_data.astype("float32"))
index = faiss.IndexIDMap(faiss.IndexFlatIP(1024))
index.add_with_ids(encoded_data, np.array(range(0, len(vacancies))))
faiss.write_index(index, "vacancy_metrics.index")

In [187]:
def get_results_by_query(query, top_k: int = 5):
  results = search(query, top_k=top_k, index=index, model=model)
  model_inputs = [[query, item["text"]] for item in results]
  scores = cross_score(model_inputs)
  ranked_results = [
      {"Title": inp["title"], "Text": inp["text"], "Score": score, 'inner_idx': inp['vacancy_idx']}
      for inp, score in zip(results, scores)
  ]
  ranked_results = sorted(ranked_results, key=lambda x: x["Score"], reverse=True)
  return ranked_results

Посчитаем результаты. Будем предсказывать топ10 вакансий по каждому резюме

In [194]:
result_dict = defaultdict(list)
for idx, row in resumes.iterrows():
  resume_inx = top100_resume.index[top100_resume.text == row['text']].tolist()[0]
  top_k = 10
  ranked_results = get_results_by_query(row['text'][:1024], top_k = top_k)
  ranked_results_internal_vac_idx = [vac['inner_idx'] for vac in ranked_results]
  internal_vac_texts = vacancies.iloc[ranked_results_internal_vac_idx]['text'].tolist()
  external_vac_indexes = top100_vacancies.index[top100_vacancies.text.isin(internal_vac_texts)].tolist()
  result_dict[resume_inx] = external_vac_indexes

>>>> Results in Total Time: 0.1343836784362793
>>>> Results in Total Time: 0.09002304077148438
>>>> Results in Total Time: 0.11592626571655273
>>>> Results in Total Time: 0.1261446475982666
>>>> Results in Total Time: 0.10978937149047852
>>>> Results in Total Time: 0.10180115699768066
>>>> Results in Total Time: 0.0744316577911377
>>>> Results in Total Time: 0.08184504508972168
>>>> Results in Total Time: 0.05741691589355469
>>>> Results in Total Time: 0.07215285301208496
>>>> Results in Total Time: 0.052852630615234375
>>>> Results in Total Time: 0.0915677547454834
>>>> Results in Total Time: 0.07047319412231445
>>>> Results in Total Time: 0.0752418041229248
>>>> Results in Total Time: 0.043642282485961914
>>>> Results in Total Time: 0.06479501724243164


In [195]:
labeling_dict = defaultdict(set)
for idx, row in labeling.iterrows():
  labeling_dict[row['resume_index']].add(row['vacancy_index'])

In [196]:
def precision(groundtruth,predicted):
   return len(set(predicted).intersection(set(groundtruth))) / len(predicted)
def recall(groundruth, predicted):
   return len(set(predicted).intersection(set(groundtruth))) / len(groundtruth)

def f1(precision, recall):
  return (2 * (precision * recall) / (precision + recall)) if (precision + recall) > 0 else 0

In [199]:
precisions, recalls, f1s = list(), list(), list()
for resume_idx, vacancy_idxs in result_dict.items():
  predicted   = vacancy_idxs
  groundtruth = labeling_dict[resume_idx]
  prec = precision(groundtruth, predicted)
  rec = recall(groundtruth, predicted)
  f1s.append(f1(prec, rec))
  precisions.append(prec)
  recalls.append(rec)

print(f"Mean precision: {sum(precisions) / len(precisions)}")
print(f"Mean recall: {sum(recalls) / len(recalls)}")
print(f"Mean F1: {sum(f1s) / len(f1s)}")

Mean precision: 0.16875000000000004
Mean recall: 0.1173611111111111
Mean F1: 0.13794258373205742


## Вывод

Нас устраивает качество работы алгоритма, поэтому возьмем его в качестве рабочего и будем выводить его в продуктив